In [1]:
import cv2
import numpy as np
import os
from utils import *
import matplotlib.pyplot as plt


 ### 🎯 Filtro de paso alto utilizando suavizado Gaussiano

Esta función `filtro_paso_alto_gaussiano` aplica un filtro de paso alto a una imagen en escala de grises. El objetivo es resaltar los detalles finos (alta frecuencia), como bordes y texturas.

**¿Cómo funciona?**
- Primero suaviza la imagen con un filtro Gaussiano (paso bajo).
- Luego, resta esta versión suavizada de la imagen original. Esto elimina las bajas frecuencias (zonas uniformes) y conserva las altas (bordes y detalles).

Este tipo de filtro es útil para análisis de nitidez, detección de bordes o estudios de textura.

In [2]:
def filtro_paso_alto_gaussiano(img_gray, ksize=5):
    suavizada = cv2.GaussianBlur(img_gray, (ksize, ksize), 0)
    return cv2.subtract(img_gray, suavizada)


### 🗂️ Procesamiento masivo de imágenes con filtro Gaussiano

Esta función `procesar_imagenes_carpeta_gaussiano` aplica el filtro de paso alto Gaussiano a todas las imágenes `.png` dentro de una estructura de carpetas.

**Qué hace:**
- Recorre subcarpetas dentro de una carpeta principal.
- Aplica el filtro de paso alto a cada imagen.
- Guarda las imágenes filtradas manteniendo la estructura original.
- Calcula la varianza de cada imagen procesada y la guarda en un archivo `.txt`.

**Utilidad:**
Permite automatizar el análisis de alta frecuencia en un conjunto grande de imágenes, útil por ejemplo en evaluaciones de nitidez o comparativas de calidad visual.


In [3]:
def procesar_imagenes_carpeta_gaussiano(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
    ksize=5
):
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)

        if not os.path.isdir(ruta_subcarpeta):
            continue

        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta, "gaussiano")
        os.makedirs(carpeta_salida_sub, exist_ok=True)

        ruta_txt = os.path.join(carpeta_salida_sub, "info_varianzas_gaussiano.txt")
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Varianzas de alta frecuencia (filtro gaussiano)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("La varianza indica la dispersión de los valores de alta frecuencia.\n")
            archivo_txt.write("Valores mayores suelen corresponder a más bordes y mayor nitidez.\n\n")

            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if not filename.lower().endswith(".png"):
                    continue

                ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                img_bgr = cv2.imread(ruta_imagen_entrada)
                if img_bgr is None:
                    continue

                img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

                # Aplicar filtro y calcular varianza en una sola vez
                altas_frec_img = filtro_paso_alto_gaussiano(img_gray, ksize=ksize)
                varianza = np.var(altas_frec_img)

                # Normalizar imagen para guardarla con buen contraste
                hf_min, hf_max = altas_frec_img.min(), altas_frec_img.max()
                if hf_max > hf_min:
                    altas_frec_escalada = ((altas_frec_img - hf_min) / (hf_max - hf_min) * 255).astype(np.uint8)
                else:
                    altas_frec_escalada = altas_frec_img.astype(np.uint8)

                nombre, _ = os.path.splitext(filename)
                nombre_salida = f"{nombre}_alta_frecuencia_gaussiano.png"
                ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)

                cv2.imwrite(ruta_imagen_salida, altas_frec_escalada)
                archivo_txt.write(f"{filename} -> varianza: {varianza:.3f}\n")

    print("Procesamiento completado con filtro Gaussiano.")


In [4]:
# Ejecuta el procesamiento

procesar_imagenes_carpeta_gaussiano(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
    ksize=5
)

Procesamiento completado con filtro Gaussiano.
